In [ ]:

# Quantum computing framework
!pip install pennylane
!pip install langchain

# PyTorch ecosystem
!pip install torch torchvision torchaudio

# Hugging Face ecosystem
!pip install transformers datasets accelerate

# Fine-tuning and optimization libraries
!pip install peft bitsandbytes trl

# Hugging Face Hub
!pip install huggingface_hub

# Sentence transformers for embeddings
!pip install sentence-transformers

# Machine learning utilities
!pip install scikit-learn

# Numerical computing
!pip install numpy
import torch
import gc

# Clear GPU memory before starting
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    # Force garbage collection
    gc.collect()
import os
import re
import gc
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline as hf_pipeline
)
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pennylane as qml
from pennylane import numpy as qnp
from pennylane.qnn import TorchLayer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# Quantum Configuration
N_QUBITS = 4
QUANTUM_DEVICE = qml.device("default.qubit", wires=N_QUBITS)

# Emotion labels for classification
EMOTION_LABELS = ['neutral', 'sadness', 'anxiety', 'anger', 'hopelessness', 'loneliness']

class QuantumEmotionClassifier(nn.Module):
    """Quantum-enhanced emotion classifier using PennyLane"""
    
    def __init__(self, n_qubits=N_QUBITS, n_classes=len(EMOTION_LABELS)):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_classes = n_classes
        # Add emotion-to-prompt mapping
        
        # Classical preprocessing layers
        self.classical_preprocessing = nn.Sequential(
            nn.Linear(384, 64),  # From sentence transformer to reduced dimension
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, n_qubits),
            nn.Tanh()  # Normalize to [-1, 1] for quantum encoding
        )
        
        # Quantum layer
        self.quantum_layer = self._create_quantum_layer()
        
        # Classical postprocessing
        self.classical_postprocessing = nn.Sequential(
            nn.Linear(n_qubits, 32),
            nn.ReLU(),  
            nn.Dropout(0.3),
            nn.Linear(32, n_classes)
        )
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize quantum circuit weights properly"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_uniform_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
        
    def _create_quantum_layer(self):
        """Create the quantum neural network layer"""
        
        def feature_map(x, wires):
            """Encode classical data into quantum states"""
            for i in range(len(wires)):
                qml.RY(x[i], wires=wires[i])
                
        def variational_circuit(weights, wires):
            """Parameterized quantum circuit for learning"""
            # Layer 1: Single qubit rotations
            for i in range(len(wires)):
                qml.RY(weights[0, i], wires=wires[i])
                qml.RZ(weights[1, i], wires=wires[i])
            
            # Layer 2: Entangling gates
            for i in range(len(wires) - 1):
                qml.CNOT(wires=[wires[i], wires[i+1]])
            
            # Layer 3: More rotations
            for i in range(len(wires)):
                qml.RY(weights[2, i], wires=wires[i])
                
        @qml.qnode(QUANTUM_DEVICE, interface="torch")
        def quantum_circuit(inputs, weights):
            # Encode classical data
            feature_map(inputs, wires=range(self.n_qubits))
            
            # Apply variational circuit
            variational_circuit(weights, wires=range(self.n_qubits))
            
            # Measure expectation values
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
        
        # Fixed weight shapes - use proper tensor dimensions
        weight_shapes = {"weights": [3, self.n_qubits]}
        
        return TorchLayer(quantum_circuit, weight_shapes)
    
    def forward(self, x):
        x = self.classical_preprocessing(x)
        x = self.quantum_layer(x)
        x = x.reshape(-1, self.n_qubits)  # Ensure 2D shape
        x = self.classical_postprocessing(x)
        return torch.softmax(x, dim=1)

class EmotionDatasetGenerator:
    """Generate synthetic emotion dataset for training"""
    
    def __init__(self):
        self.goemotions_path = goemotions_path
        self.twitter_path = twitter_path
        self.emotion_examples = {
            'neutral': [
                "How are you today?",
                "I went to the store.",
                "The weather is nice.",
                "I finished my work.",
                "Let's have lunch."
            ],
            'sadness': [
                "I feel so down today.",
                "Everything seems hopeless.",
                "I can't stop crying.",
                "I feel empty inside.",
                "Nothing makes me happy anymore.",
                "I'm feeling really low.",
                "Life feels meaningless right now."
            ],
            'anxiety': [
                "I'm so worried about everything.",
                "I can't stop feeling nervous.",
                "My heart is racing with fear.",
                "I'm panicking about the future.",
                "I feel so stressed and overwhelmed.",
                "I'm afraid something bad will happen.",
                "I can't calm my anxious thoughts."
            ],
            'anger': [
                "I'm so frustrated right now.",
                "Everything is making me angry.",
                "I'm furious about this situation.",
                "I feel so irritated and mad.",
                "This is driving me crazy with rage.",
                "I'm really upset and angry.",
                "I can't control my anger anymore."
            ],
            'hopelessness': [
                "Nothing will ever get better.",
                "I feel completely hopeless.",
                "There's no point in trying anymore.",
                "I've given up on everything.",
                "I don't see any way out.",
                "Everything feels pointless.",
                "I have no hope left."
            ],
            'loneliness': [
                "I feel so alone in this world.",
                "Nobody understands me.",
                "I have no one to talk to.",
                "I feel completely isolated.",
                "Everyone has abandoned me.",
                "I'm all by myself.",
                "I feel so lonely and forgotten."
            ]
        }
    def load_goemotions(path='/kaggle/input/goemotions-basic-eda'):
        # Download from https://github.com/google-research/google-research/tree/master/goemotions/data
        # Columns: 'text', 'labels'
        df = pd.read_csv(path)
        # GoEmotions has 27 emotions + neutral, multi-label per sample
        # Map GoEmotions fine labels to your 6-class taxonomy
        goemotions_map = {
            'neutral': 'neutral',
            'sadness': 'sadness',
            'fear': 'anxiety',
            'anger': 'anger',
            'disgust': 'hopelessness',
            'loneliness': 'loneliness',
            # All other emotions → you may map or ignore as needed
        }
        texts, labels = [], []
        for _, row in df.iterrows():
            label_list = eval(row['labels']) if isinstance(row['labels'], str) else row['labels']
            for label in label_list:
                mapped = goemotions_map.get(label)
                if mapped:
                    texts.append(row['text'])
                    labels.append(mapped)
                    break  # Use the first mapped label for this sample
        return texts, labels
    
    def load_twitter_emotion(path='/kaggle/input/emotions-dataset-for-nlp'):
        # Download from https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp
        df = pd.read_csv(path)
        twitter_map = {
            'sadness': 'sadness',
            'joy': 'neutral',  # Map joy to neutral or ignore
            'love': 'neutral', # Map to neutral or ignore
            'anger': 'anger',
            'fear': 'anxiety',
            'surprise': 'neutral', # Map to neutral or ignore
        }
        texts, labels = [], []
        for _, row in df.iterrows():
            mapped = twitter_map.get(row['emotion'])
            if mapped:
                texts.append(row['text'])
                labels.append(mapped)
        return texts, labels

    def generate_dataset(self, samples_per_emotion=50):
        """Generate a balanced dataset of emotion examples"""
        texts = []
        labels = []
        
        for emotion, examples in self.emotion_examples.items():
            # Repeat and slightly modify examples to reach desired sample count
            current_examples = examples.copy()
            while len(current_examples) < samples_per_emotion:
                # Add variations of existing examples
                base_examples = examples.copy()
                for example in base_examples:
                    if len(current_examples) >= samples_per_emotion:
                        break
                    # Simple variation by adding prefixes
                    variations = [
                        f"I really {example.lower()}",
                        f"Today {example.lower()}",
                        f"Right now {example.lower()}"
                    ]
                    current_examples.extend(variations)
            
            # Take exactly the number we need
            current_examples = current_examples[:samples_per_emotion]
            texts.extend(current_examples)
            labels.extend([emotion] * len(current_examples))
        go_texts, go_labels = load_goemotions(self.goemotions_path)
        tw_texts, tw_labels = load_twitter_emotion(self.twitter_path)
        texts += go_texts + tw_texts
        labels += go_labels + tw_labels

        # Balance the dataset for each emotion
        df = pd.DataFrame({'text': texts, 'label': labels})
        balanced = []
        for emotion in EMOTION_LABELS:
            group = df[df['label'] == emotion]
            if len(group) > samples_per_emotion:
                group = group.sample(samples_per_emotion, random_state=42)
            balanced.append(group)
        balanced_df = pd.concat(balanced)
        return balanced_df['text'].tolist(), balanced_df['label'].tolist()

def get_device_config():
    """Check available hardware and return appropriate configuration"""
    try:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gpu_memory = torch.cuda.get_device_properties(0).total_memory
            free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
            
            print(f"CUDA available: {torch.cuda.get_device_name(0)}")
            print(f"Total GPU memory: {gpu_memory / 1e9:.2f} GB")
            print(f"Free GPU memory: {free_memory / 1e9:.2f} GB")
            
            return {
                "device": "cuda",
                "use_cuda": True,
                "load_in_4bit": free_memory > 1e9
            }
        else:
            print("CUDA not available, using CPU")
            return {
                "device": "cpu",
                "use_cuda": False,
                "load_in_4bit": False
            }
    except Exception as e:
        print(f"Device configuration error: {e}")
        return {
            "device": "cpu",
            "use_cuda": False,
            "load_in_4bit": False
        }

def load_classical_model(model_path="aboonaji/llama2finetune-v2"):
    """Load the classical language model with optimizations"""
    torch.cuda.empty_cache()
    gc.collect()
    try:
        config = get_device_config()
        hf_token = os.environ.get('HUGGINGFACE_TOKEN', None)
        token_kwargs = {"token": hf_token} if hf_token else {}
        
        # Load tokenizer
        try:
            tokenizer = AutoTokenizer.from_pretrained(
                model_path,
                trust_remote_code=True,
                max_length=512,
                **token_kwargs
            )
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.padding_side = "right"
        except Exception as e:
            print(f"Error loading tokenizer: {e}")
            return None, None
        
        # Load model
        try:
            if config["use_cuda"] and config["load_in_4bit"]:
                  quantization_config = BitsAndBytesConfig(
                      load_in_4bit=True,
                      bnb_4bit_compute_dtype=torch.float16,
                      bnb_4bit_quant_type="nf4",
                      bnb_4bit_use_double_quant=True
                  )
                  
                  model = AutoModelForCausalLM.from_pretrained(
                      model_path,
                      quantization_config=quantization_config,
                      device_map="auto",
                      low_cpu_mem_usage=True,
                      torch_dtype=torch.float16,
                      max_memory={0: "2GB"},
                      **token_kwargs
                  )
            else:
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map=config["device"],
                    torch_dtype=torch.float16 if config["use_cuda"] else torch.float32,
                    low_cpu_mem_usage=True,
                    **token_kwargs
                )
            
            return model, tokenizer
            
        except Exception as e:
            print(f"Error loading model: {e}")
            return None, None
            
    except Exception as e:
        print(f"Unexpected error in load_classical_model: {e}")
        return None, None

class QuantumEmotionalSupportBot:
    """Quantum-enhanced emotional support bot"""
    
    def __init__(self, classical_model=None, tokenizer=None, train_quantum=True):
        self.classical_model = classical_model
        self.tokenizer = tokenizer
        self.is_dummy_mode = False
        self.emotion_prompts = {
            'neutral': "Respond supportively while maintaining professionalism",
            'sadness': "Acknowledge the sadness and offer compassionate support",
            'anxiety': "Respond empathetically with reassurance and calm",
            'anger': "Respond patiently to de-escalate tension",
            'hopelessness': "Offer hope and emphasize their strength",
            'loneliness': "Express understanding and emphasize connection"
        }
        # Initialize sentence transformer for embeddings
        print("Loading sentence transformer...")
        try:
            self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')
            print("Sentence transformer loaded successfully")
            if torch.cuda.is_available():
              torch.cuda.empty_cache()
              gc.collect()
        except Exception as e:
            print(f"Error loading sentence transformer: {e}")
            self.sentence_encoder = None
        
        # Initialize quantum emotion classifier
        print("Initializing quantum emotion classifier...")
        try:
            self.quantum_classifier = QuantumEmotionClassifier()
            self.label_encoder = LabelEncoder()
            self.label_encoder.fit(EMOTION_LABELS)
            print("Quantum emotion classifier initialized successfully")
        except Exception as e:
            print(f"Error initializing quantum classifier: {e}")
            raise e
        
        # Train quantum classifier if requested
        if train_quantum and self.sentence_encoder:
            try:
                self.train_quantum_classifier()
            except Exception as e:
                print(f"Error training quantum classifier: {e}")
                print("Continuing without quantum training...")
        
        # Load classical model if not provided
        if classical_model is None or tokenizer is None:
            print("Loading classical language model...")
            self.classical_model, self.tokenizer = load_classical_model()
            
            if self.classical_model is None:
                print("Classical model loading failed. Using dummy mode.")
                self._initialize_dummy_mode()
                return
        
        # Initialize text generation pipeline
        if not self.is_dummy_mode:
            try:
                config = get_device_config()
                max_new_tokens = 128 if config["use_cuda"] else 64
                
                self.text_generation_pipeline = hf_pipeline(
                    task="text-generation",
                    model=self.classical_model,
                    tokenizer=self.tokenizer,
                    max_new_tokens=max_new_tokens,
                    truncation=True,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    repetition_penalty=1.2,
                    device_map="auto" if config["use_cuda"] else None,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            except Exception as e:
                print(f"Error initializing pipeline: {e}")
                self._initialize_dummy_mode()
                return
        
        self.memories = {}  # {user_id: ConversationBufferMemory}
        self.user_emotions = {}
        # Safety guardrails
        self.crisis_keywords = ["suicide", "kill myself", "end my life", "want to die"]
        self.crisis_response = (
            "I notice you've mentioned something that sounds serious. If you're having thoughts of "
            "harming yourself, please know that you're not alone and support is available. "
            "Please consider talking to a mental health professional or calling a crisis helpline:\n"
            "- National Suicide Prevention Lifeline: 988\n"
            "- Crisis Text Line: Text HOME to 741741\n\n"
            "Would you like to talk more about what you're experiencing? I'm here to listen."
        )
        
        print("Quantum EmotionalSupportBot initialized successfully!")
    def get_emotion_instruction(self, emotion, confidence):
        base = self.emotion_prompts.get(emotion, "")
        if confidence < 0.5:
            return base + " while acknowledging uncertainty"
        elif confidence > 0.8:
            return base + " with strong conviction"
        return base
    def evaluate_emotion_classifier(model, data_loader, label_encoder, device='cpu'):
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
    
        # Metrics calculation
        acc = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        cm = confusion_matrix(all_labels, all_preds)
        
        print("Evaluation Metrics:")
        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1-score:  {f1:.4f}")
        print("Confusion Matrix:")
        print(cm)
        return {
            "accuracy": acc,
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "confusion_matrix": cm
        }
    def train_quantum_classifier(self, epochs=50, batch_size=32):
        """Train the quantum emotion classifier"""
        print("Generating training data...")
        dataset_generator = EmotionDatasetGenerator(
                goemotions_path='/kaggle/input/goemotions-basic-eda',
                twitter_path='/kaggle/input/emotions-dataset-for-nlp',
            )       
        texts, labels = dataset_generator.generate_dataset(samples_per_emotion=200)
        
        print(f"Generated {len(texts)} training samples")
        
        # Encode texts using sentence transformer
        print("Encoding texts...")
        try:
            embeddings = self.sentence_encoder.encode(texts, convert_to_tensor=True)
            print(f"Embeddings shape: {embeddings.shape}")
        except Exception as e:
            print(f"Error encoding texts: {e}")
            return
        
        # Encode labels
        encoded_labels = self.label_encoder.transform(labels)
        labels_tensor = torch.tensor(encoded_labels, dtype=torch.long)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            embeddings, labels_tensor, test_size=0.2, random_state=42, stratify=encoded_labels
        )
        
        # Get device and move everything to the same device
        device = get_device_config()["device"]
        self.quantum_classifier.to(device)
        
        # Convert to proper device and ensure proper tensor types
        X_train = X_train.to(device).float()
        X_test = X_test.to(device).float()
        y_train = y_train.to(device).long()
        y_test = y_test.to(device).long()
        
        # Create data loaders
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        # Training setup
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.quantum_classifier.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
        
        print(f"Training quantum classifier on {device}...")
        
        # Training loop
        for epoch in range(epochs):
            self.quantum_classifier.train()
            total_loss = 0
            correct = 0
            total = 0
            
            for batch_embeddings, batch_labels in train_loader:
                # Ensure tensors are on the correct device
                batch_embeddings = batch_embeddings.to(device).float()
                batch_labels = batch_labels.to(device).long()
                
                optimizer.zero_grad()
                outputs = self.quantum_classifier(batch_embeddings)
                loss = criterion(outputs, batch_labels)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += batch_labels.size(0)
                correct += (predicted == batch_labels).sum().item()
            
            scheduler.step()
            
            # Validation
            if (epoch + 1) % 10 == 0:
                self.quantum_classifier.eval()
                val_correct = 0
                val_total = 0
                with torch.no_grad():
                    for batch_embeddings, batch_labels in test_loader:
                        batch_embeddings = batch_embeddings.to(device).float()
                        batch_labels = batch_labels.to(device).long()
                        outputs = self.quantum_classifier(batch_embeddings)
                        _, predicted = torch.max(outputs.data, 1)
                        val_total += batch_labels.size(0)
                        val_correct += (predicted == batch_labels).sum().item()
                
                print(f'Epoch [{epoch+1}/{epochs}], '
                    f'Loss: {total_loss/len(train_loader):.4f}, '
                    f'Train Acc: {100*correct/total:.2f}%, '
                    f'Val Acc: {100*val_correct/val_total:.2f}%')
        
        print("Quantum classifier training completed!")
        # --- EVALUATION METRICS ---
        metrics = self.evaluate_emotion_classifier(
            model=self.quantum_classifier,
            data_loader=test_loader,
            label_encoder=self.label_encoder,
            device=get_device_config()["device"]
        )
        print("Final Evaluation Metrics:")
        for k, v in metrics.items():
            print(f"{k}: {v}")
        
        # Save the trained model
        try:
            torch.save({
                'model_state_dict': self.quantum_classifier.state_dict(),
                'label_encoder': self.label_encoder
            }, 'quantum_emotion_classifier.pth')
            print("Quantum model saved to 'quantum_emotion_classifier.pth'")
        except Exception as e:
            print(f"Error saving quantum model: {e}")
    
    def detect_emotion_quantum(self, text):
        if self.sentence_encoder is None or not hasattr(self, 'quantum_classifier'):
            return self.detect_emotion_classical(text)
        try:
            embedding = self.sentence_encoder.encode([text], convert_to_tensor=True)
            if embedding.ndim == 1:
                embedding = embedding.unsqueeze(0)
            if embedding.shape[0] == 0 or embedding.shape[1] == 0:
                return "neutral", 0.5
    
            device = next(self.quantum_classifier.parameters()).device
            embedding = embedding.to(device)
            self.quantum_classifier.eval()
            with torch.no_grad():
                prediction = self.quantum_classifier(embedding)
                if prediction.ndim == 1:
                    prediction = prediction.unsqueeze(0)
                if prediction.shape[0] == 0 or prediction.shape[1] == 0:
                    return "neutral", 0.5
                predicted_indices = torch.argmax(prediction, dim=1).cpu().numpy()
                if predicted_indices.size == 0:
                    return "neutral", 0.5
                try:
                    predicted_class = int(predicted_indices[0])
                except Exception:
                    return "neutral", 0.5
                confidence = float(torch.max(prediction).item())
            emotion = self.label_encoder.inverse_transform([predicted_class])[0]
            if confidence > 0.3:
                return emotion, confidence
            else:
                return "neutral", confidence
        except Exception as e:
            # Optionally log the error for debugging
            return self.detect_emotion_classical(text)



    # Step 2: Use emotion history for more nuanced responses
    def build_emotion_context(self, user_id):
        """Build emotional context from user's emotion history"""
        if user_id not in self.user_emotions or not self.user_emotions[user_id]:
            return ""
        
        recent_emotions = self.user_emotions[user_id][-3:]  # Last 3 emotions
        emotion_context = []
        
        # Analyze emotion patterns
        emotions_list = [e['emotion'] for e in recent_emotions]
        confidences = [e['confidence'] for e in recent_emotions]
        
        # Check for persistent patterns
        if len(set(emotions_list)) == 1 and emotions_list[0] != 'neutral':
            emotion_context.append(f"User has been consistently feeling {emotions_list[0]}")
        elif len(emotions_list) >= 2:
            if emotions_list[-1] != emotions_list[-2]:
                emotion_context.append(f"User's emotion shifted from {emotions_list[-2]} to {emotions_list[-1]}")
        
        # Check for intensity changes
        if len(confidences) >= 2:
            if confidences[-1] > confidences[-2] + 0.2:
                emotion_context.append("Emotional intensity has increased")
            elif confidences[-1] < confidences[-2] - 0.2:
                emotion_context.append("Emotional intensity has decreased")
        
        # Check for concerning patterns
        negative_emotions = ['sadness', 'anxiety', 'anger', 'hopelessness', 'loneliness']
        if all(e in negative_emotions for e in emotions_list):
            emotion_context.append("User has shown persistent negative emotions")
        
        return "; ".join(emotion_context) if emotion_context else ""

    # Step 3: Reference previous topics mentioned
    def extract_and_store_topics(self, user_input, user_id):
        """Extract and store key topics from user messages"""
        if not hasattr(self, 'user_topics'):
            self.user_topics = {}
        
        if user_id not in self.user_topics:
            self.user_topics[user_id] = []
        
        # Simple topic extraction (could be enhanced with NLP)
        topics = []
        
        # Personal references
        personal_keywords = {
            'work': ['work', 'job', 'career', 'office', 'boss', 'colleague', 'employment'],
            'family': ['family', 'parents', 'mom', 'dad', 'mother', 'father', 'sibling', 'brother', 'sister'],
            'relationship': ['boyfriend', 'girlfriend', 'partner', 'spouse', 'husband', 'wife', 'relationship', 'dating'],
            'health': ['health', 'doctor', 'medicine', 'illness', 'pain', 'therapy', 'treatment'],
            'school': ['school', 'college', 'university', 'student', 'exam', 'grade', 'homework', 'class'],
            'friends': ['friend', 'friends', 'social', 'party', 'hangout', 'buddy'],
            'money': ['money', 'financial', 'bills', 'debt', 'income', 'salary', 'budget', 'broke']
        }
        
        user_input_lower = user_input.lower()
        for topic, keywords in personal_keywords.items():
            if any(keyword in user_input_lower for keyword in keywords):
                topics.append(topic)
        
        # Store topics with timestamp
        for topic in topics:
            if topic not in [t['topic'] for t in self.user_topics[user_id][-5:]]:  # Avoid duplicates
                self.user_topics[user_id].append({
                    'topic': topic,
                    'timestamp': len(self.user_topics[user_id]),
                    'context': user_input[:100]  # Store snippet for context
                })
        
        # Keep only recent topics
        if len(self.user_topics[user_id]) > 10:
            self.user_topics[user_id] = self.user_topics[user_id][-10:]
    def get_smoothed_emotion(self, user_id, new_emotion, new_confidence):
        history = self.user_emotions.get(user_id, [])
        if history:
            last = history[-1]
            if new_confidence < 0.6 and last['confidence'] > 0.6:
                return last['emotion'], last['confidence']
        return new_emotion, new_confidence

    def build_topic_context(self, user_id):
        """Build context from previously mentioned topics"""
        # Check if user_topics attribute and user_id exist
        if not hasattr(self, 'user_topics') or user_id not in self.user_topics:
            return ""
        
        # Get the user's topic history
        topic_history = self.user_topics[user_id]
        if not topic_history:
            return ""
        
        # Collect unique topics in reverse order (most recent first)
        seen = set()
        unique_recent_topics = []
        for topic_entry in reversed(topic_history):
            topic = topic_entry['topic']
            if topic not in seen:
                unique_recent_topics.append(topic)
                seen.add(topic)
            if len(unique_recent_topics) == 3:
                break
        
        if not unique_recent_topics:
            return ""
        
        # Reverse to maintain chronological order (oldest to newest)
        unique_recent_topics.reverse()
        return f"Previous topics discussed: {', '.join(unique_recent_topics)}"




            
    def generate_response_with_history(self, user_input, user_id="default_user"):
        """Generate response with full conversation and emotional history"""
        try:
            # Safety check
            safety_response = self.check_safety(user_input)
            if safety_response:
                return safety_response
    
            # Extract and store topics from current input
            self.extract_and_store_topics(user_input, user_id)
    
            # Emotion detection
            try:
                if hasattr(self, 'quantum_classifier'):
                    emotion, confidence = self.detect_emotion_quantum(user_input)
                    #print(f"Quantum emotion detection: {emotion} (confidence: {confidence:.3f})")
                else:
                    emotion, confidence = self.detect_emotion_classical(user_input)
                    #print(f"Classical emotion detection: {emotion} (confidence: {confidence:.3f})")
            except Exception as e:
                print(f"Error in emotion detection: {e}")
                emotion, confidence = ("neutral", 0.5)
            emotion, confidence = self.get_smoothed_emotion(user_id, emotion, confidence)

            # Track emotion
            self.track_emotion(user_id, emotion, confidence)
    
            # Build comprehensive context
            from langchain.schema import HumanMessage, AIMessage
            memory = self.get_memory(user_id)
            chat_history = memory.load_memory_variables({})["chat_history"]
    
            context_parts = []
            for msg in chat_history[-5:]:
                if isinstance(msg, HumanMessage):
                    context_parts.append(f"User: {msg.content}")
                elif isinstance(msg, AIMessage):
                    context_parts.append(f"Bot: {msg.content}")
    
            # Emotional and topic context
            emotion_context = self.build_emotion_context(user_id)
            if emotion_context:
                context_parts.append(f"Emotional context: {emotion_context}")
            topic_context = self.build_topic_context(user_id)
            if topic_context:
                context_parts.append(topic_context)
    
            emotion_instruction = self.get_emotion_instruction(emotion, confidence)

    
            # Build the full prompt
            if context_parts:
                full_context = "\n".join(context_parts)
                formatted_input = (
                    f"<s>[INST] Context: {full_context}\n"
                    f"Emotional State: {emotion} (confidence: {confidence:.2f})\n"
                    f"Instruction: {emotion_instruction}\n"
                    f"Current message: {user_input} [/INST]"
                )
            else:
                formatted_input = (
                    f"<s>[INST] Emotional State: {emotion} (confidence: {confidence:.2f})\n"
                    f"Instruction: {emotion_instruction}\n"
                    f"Current message: {user_input} [/INST]"
                )
    
            # Generate response
            try:
                if self.is_dummy_mode:
                    response = self.text_generation_pipeline(formatted_input)[0]['generated_text']
                else:
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    with torch.no_grad():
                        response = self.text_generation_pipeline(
                            formatted_input,
                            return_full_text=False,
                            max_new_tokens=150,
                            temperature=0.7,
                            top_p=0.9
                        )[0]['generated_text']
            except Exception as e:
                print(f"Error in response generation: {e}")
                return "I'm having trouble processing that right now. Could you rephrase or try again?"
    
            # Clean and enhance response
            clean_response = self.postprocess_response(response)
            enhanced_response = self.enhance_response_with_history(clean_response, user_id, emotion)
    
            return enhanced_response
    
        except Exception as e:
            print(f"Critical error in generate_response_with_history: {e}")
            return "I'm experiencing some technical difficulties. Could you share a bit more about what you're feeling?"

    def has_positive_shift(self, user_id):
        history = self.user_emotions.get(user_id, [])
        if len(history) >= 2:
            prev, curr = history[-2], history[-1]
            return prev['emotion'] != 'neutral' and curr['emotion'] == 'neutral' and curr['confidence'] > 0.6
        return False

    def enhance_response_with_history(self, response, user_id, current_emotion):
        """Add personalized touches based on conversation history"""
        enhancements = []
        
        # Check for emotional progress
        if user_id in self.user_emotions and len(self.user_emotions[user_id]) >= 2:
            prev_emotion = self.user_emotions[user_id][-2]['emotion']
            
            if self.has_positive_shift(user_id):
                enhancements.append("I'm glad to hear you're feeling a bit better than before.")

        # Reference recurring topics
        if hasattr(self, 'user_topics') and user_id in self.user_topics:
            recent_topics = [t['topic'] for t in self.user_topics[user_id][-3:]]
            topic_counts = {}
            for topic in recent_topics:
                topic_counts[topic] = topic_counts.get(topic, 0) + 1
            
            recurring_topics = [topic for topic, count in topic_counts.items() if count >= 2]
            if recurring_topics:
                if 'work' in recurring_topics:
                    enhancements.append("I know work has been on your mind lately.")
                elif 'family' in recurring_topics:
                    enhancements.append("Family situations can be really complex.")
                elif 'relationship' in recurring_topics:
                    enhancements.append("Relationships can bring up so many different feelings.")
        
        # Check for persistent negative patterns
        trend = self.get_emotion_trend(user_id)
        if trend == "persistent_negative":
            enhancements.append("I've noticed you've been going through a particularly challenging time. Please remember that support is available, and you don't have to go through this alone.")
        
        # Combine response with enhancements
        if enhancements:
            return response + " " + " ".join(enhancements)
        
        return response

    # Additional helper method for topic analysis
    def get_topic_summary(self, user_id):
        """Get a summary of topics the user has discussed"""
        if not hasattr(self, 'user_topics') or user_id not in self.user_topics:
            return "No previous topics discussed"
        
        topics = [t['topic'] for t in self.user_topics[user_id]]
        topic_counts = {}
        for topic in topics:
            topic_counts[topic] = topic_counts.get(topic, 0) + 1
        
        # Sort by frequency
        sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
        
        summary = []
        for topic, count in sorted_topics[:5]:  # Top 5 topics
            summary.append(f"{topic} ({count} times)")
        
        return "; ".join(summary)
    def get_memory(self, user_id):
        """Get or create memory for a specific user"""
        if user_id not in self.memories:
            self.memories[user_id] = ConversationBufferMemory(
                return_messages=True,
                memory_key="chat_history",
                input_key="input",
                output_key="output"
            )
        return self.memories[user_id]
    # Enhanced debug info to show history context
    def display_debug_info_enhanced(self, user_id="default_user"):
        """Enhanced debug information including history analysis"""
        print("\n--- Quantum EmotionalSupportBot Enhanced Debug Info ---")
        
        # Call original debug info
        self.display_debug_info(user_id)
        
        # Additional history-aware debug info
        print("=== CONVERSATION HISTORY ANALYSIS ===")
        
        memory = self.get_memory(user_id)
        chat_history = memory.load_memory_variables({})["chat_history"]
        print(f"Message history ({len(chat_history)} messages):")
        for i, msg in enumerate(chat_history[-5:]):
            content_preview = msg.content[:50] + '...' if len(msg.content) > 50 else msg.content
            print(f"  {i+1}. [{type(msg).__name__}] {content_preview}")

        # Emotion progression
        if user_id in self.user_emotions:
            emotions = [e['emotion'] for e in self.user_emotions[user_id]]
            print(f"Emotion progression: {' -> '.join(emotions[-5:])}")
        
        # Topic analysis
        if hasattr(self, 'user_topics') and user_id in self.user_topics:
            topic_summary = self.get_topic_summary(user_id)
            print(f"Topics discussed: {topic_summary}")
        else:
            print("No topics tracked yet")
        
        # Context building preview
        emotion_context = self.build_emotion_context(user_id)
        topic_context = self.build_topic_context(user_id)
        
        if emotion_context:
            print(f"Current emotion context: {emotion_context}")
        if topic_context:
            print(f"Current topic context: {topic_context}")
        
        print("====================================================\n")

    # Update the main chat method to use the new history-aware generation
    def chat_with_history(self, user_input, user_id="default_user"):
        """Enhanced chat interface with full history awareness"""
        try:
            
            # Clean input
            cleaned_input = re.sub(r'\[INST\]|\[/INST\]', '', user_input).strip()
            
            # Generate response with history
            response = self.generate_response_with_history(user_input, user_id)
            
            # Store user message
            
            memory = self.get_memory(user_id)
            memory.save_context({"input": cleaned_input}, {"output": response})           
            # Memory cleanup
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            return response
            
        except Exception as e:
            print(f"Error in chat_with_history method: {e}")
            return "I apologize, but I'm having some technical difficulties. Let's try again."
    
    def detect_emotion_classical(self, text):
        """Fallback classical emotion detection"""
        emotion_keywords = {
            "sadness": ["sad", "depressed", "unhappy", "miserable", "down", "blue"],
            "anxiety": ["anxious", "worried", "nervous", "stressed", "panicking", "afraid"],
            "anger": ["angry", "mad", "furious", "irritated", "annoyed", "frustrated"],
            "hopelessness": ["hopeless", "pointless", "worthless", "empty", "meaningless"],
            "loneliness": ["lonely", "alone", "isolated", "abandoned", "rejected"]
        }
        
        text_lower = text.lower()
        for emotion, keywords in emotion_keywords.items():
            if any(keyword in text_lower for keyword in keywords):
                return (emotion, 0.7)  # Return as tuple
        
        return ("neutral", 0.5)  # Return as tuple
    
    def _initialize_dummy_mode(self):
        """Initialize dummy mode for when models fail to load"""
        print("Initializing in dummy mode...")
        self.is_dummy_mode = True
        
        def dummy_generate(text, return_full_text=True, **kwargs):
            text_lower = text.lower()
            if "sad" in text_lower or "down" in text_lower:
                response = "I'm sorry to hear you're feeling down. Would you like to talk about what's contributing to these feelings?"
            elif "anxious" in text_lower or "worried" in text_lower:
                response = "Anxiety can be really challenging. Would it help to explore what's causing these feelings of worry?"
            elif "angry" in text_lower or "frustrated" in text_lower:
                response = "I can understand feeling frustrated. Sometimes anger points to something important to us."
            elif "alone" in text_lower or "lonely" in text_lower:
                response = "Feeling alone can be really difficult. Connection is so important for our wellbeing."
            else:
                response = "Thank you for sharing that with me. Would you like to tell me more about what you're experiencing?"
            
            if return_full_text:
                return [{"generated_text": text + " [/INST] " + response}]
            else:
                return [{"generated_text": response}]
        
        self.text_generation_pipeline = dummy_generate
    
    def check_safety(self, user_input):
        """Check for crisis situations"""
        input_lower = user_input.lower()
        for keyword in self.crisis_keywords:
            if keyword in input_lower:
                return self.crisis_response
        return None
    
    def track_emotion(self, user_id, emotion, confidence):
        """Track emotions with confidence scores"""
        if user_id not in self.user_emotions:
            self.user_emotions[user_id] = []
        
        self.user_emotions[user_id].append({
            'emotion': emotion,
            'confidence': confidence,
            'timestamp': len(self.user_emotions[user_id])
        })
        
        # Keep only recent emotions
        if len(self.user_emotions[user_id]) > 5:
            self.user_emotions[user_id] = self.user_emotions[user_id][-5:]
    
    def get_emotion_trend(self, user_id):
        """Analyze emotion trends"""
        if user_id not in self.user_emotions or len(self.user_emotions[user_id]) < 3:
            return None
        
        recent_emotions = [e['emotion'] for e in self.user_emotions[user_id][-3:]]
        
        # Check for persistent negative emotions
        negative_emotions = ['sadness', 'anxiety', 'anger', 'hopelessness', 'loneliness']
        persistent_negative = all(e in negative_emotions for e in recent_emotions)
        
        if persistent_negative:
            return f"persistent_negative"
        
        # Check for improvement
        if recent_emotions[-1] == "neutral" and any(e != "neutral" for e in recent_emotions[:-1]):
            return "improving"
        
        return None
    
    def preprocess_input(self, user_input):
        """Format user input with instruction tags"""
        clean_input = re.sub(r'\[INST\]|\[/INST\]', '', user_input).strip()
        return f"<s>[INST] {clean_input} [/INST]"
    
    def postprocess_response(self, response_text):
        """Clean up model response"""
        try:
            parts = response_text.split('[/INST]')
            if len(parts) > 1:
                response = parts[-1].strip()
            else:
                response = response_text.strip()
            
            response = response.replace('<s>', '').replace('</s>', '').strip()
            return response
        except Exception as e:
            print(f"Error in postprocess_response: {e}")
            return response_text.replace('[/INST]', '').replace('[INST]', '').strip()
    
    def generate_response(self, user_input, user_id="default_user"):
        """Generate response with quantum emotion awareness"""
        try:
            # Safety check
            safety_response = self.check_safety(user_input)
            if safety_response:
                return safety_response
            
            # Quantum emotion detection with proper error handling
            try:
                if hasattr(self, 'quantum_classifier'):
                    emotion, confidence = self.detect_emotion_quantum(user_input)
                    #print(f"Quantum emotion detection: {emotion} (confidence: {confidence:.3f})")
                else:
                    emotion, confidence = self.detect_emotion_classical(user_input)
                    #print(f"Classical emotion detection: {emotion} (confidence: {confidence:.3f})")
            except Exception as e:
                print(f"Error in emotion detection: {e}")
                emotion, confidence = ("neutral", 0.5)
            emotion, confidence = self.get_smoothed_emotion(user_id, emotion, confidence)

            # Track emotion
            self.track_emotion(user_id, emotion, confidence)
            
            # Format input
            formatted_input = self.preprocess_input(user_input)
            
            # Generate response with unified error handling
            try:
                if self.is_dummy_mode:
                    response = self.text_generation_pipeline(formatted_input)[0]['generated_text']
                else:
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    with torch.no_grad():
                        response = self.text_generation_pipeline(
                            formatted_input,
                            return_full_text=False
                        )[0]['generated_text']
                            
            except Exception as e:
                print(f"Error in response generation: {e}")
                return "I'm having trouble processing that right now. Could you rephrase or try again?"
            
            # Clean response
            try:
                clean_response = self.postprocess_response(response)
            except Exception as e:
                print(f"Error in response postprocessing: {e}")
                clean_response = "I'm having some trouble with my response processing. Could you try again?"
            
            # Add emotion-aware enhancements
            try:
                trend = self.get_emotion_trend(user_id)
                if trend == "persistent_negative":
                    clean_response += "\n\nI've noticed you've been going through a difficult time. Have you considered speaking with a mental health professional who might provide additional support?"
            except Exception as e:
                print(f"Error in trend analysis: {e}")
            
            return clean_response
            
        except Exception as e:
            print(f"Critical error in generate_response: {e}")
            return "I'm experiencing some technical difficulties. Could you share a bit more about what you're feeling?"

    
    def chat(self, user_input, user_id="default_user"):
        """Main chat interface"""
        try:
            
            
            # Clean input
            cleaned_input = re.sub(r'\[INST\]|\[/INST\]', '', user_input).strip()
            
            # Generate response
            response = self.generate_response(user_input, user_id)
            
            self.memory.save_context({"input": cleaned_input}, {"output": response})

            
            # Memory cleanup
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            return response
            
        except Exception as e:
            print(f"Error in chat method: {e}")
            return "I apologize, but I'm having some technical difficulties. Let's try again."
    
    def display_debug_info(self, user_id="default_user"):
        """Display debug information including quantum states"""
        print("\n--- Quantum EmotionalSupportBot Debug Info ---")
        
        # System info
        device_config = get_device_config()
        print(f"Device: {device_config['device']}")
        if device_config['use_cuda']:
            print(f"GPU: {torch.cuda.get_device_name(0)}")
        
        print(f"Running in {'dummy' if self.is_dummy_mode else 'normal'} mode")
        print(f"Quantum classifier available: {hasattr(self, 'quantum_classifier')}")
        print(f"Sentence encoder available: {self.sentence_encoder is not None}")
        
        # Emotion tracking
        if user_id in self.user_emotions:
            print(f"\nEmotion history for {user_id}:")
            for i, emotion_data in enumerate(self.user_emotions[user_id]):
                print(f"  {i+1}. {emotion_data['emotion']} (confidence: {emotion_data['confidence']:.3f})")
            
            trend = self.get_emotion_trend(user_id)
            if trend:
                print(f"Emotional trend: {trend}")
        else:
            print("No emotion data available")
        
        chat_history = self.memory.load_memory_variables({})["chat_history"]
        print(f"Message history ({len(chat_history)} messages):")
        for i, msg in enumerate(chat_history[-5:]):
            print(f"  {i+1}. {msg.content[:50]}...")
        
        print("---------------------------------------------\n")
    
    def cleanup(self):
        """Clean up resources"""
        try:
            if hasattr(self, 'classical_model') and self.classical_model and not self.is_dummy_mode:
                if hasattr(self, 'text_generation_pipeline'):
                    del self.text_generation_pipeline
                
                if torch.cuda.is_available():
                    self.classical_model.to('cpu')
                
                del self.classical_model
                self.classical_model = None
            
            if hasattr(self, 'quantum_classifier'):
                if torch.cuda.is_available():
                    self.quantum_classifier.to('cpu')
            
            # Clear memory
            self.memory.clear()

            self.user_emotions = {}
            
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            
            print("Resources cleaned up successfully")
        except Exception as e:
            print(f"Error during cleanup: {e}")

def initialize_quantum_bot(train_quantum=True):
    """Initialize the quantum-enhanced emotional support bot"""
    try:
        print("Initializing Quantum EmotionalSupportBot...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        bot = QuantumEmotionalSupportBot(train_quantum=train_quantum)
        return bot
    except Exception as e:
        print(f"Failed to initialize Quantum EmotionalSupportBot: {e}")
        return None

def run_quantum_interactive_session():
    """Run interactive session with quantum-enhanced bot"""
    # Initialize bot
    bot = initialize_quantum_bot(train_quantum=True)
    
    if not bot:
        print("Could not initialize bot. Exiting.")
        return
    
    print("\n=== Quantum EmotionalSupportBot Interactive Session ===")
    print("This bot uses quantum computing for enhanced emotion detection!")
    print("Type your messages and the bot will respond with quantum-enhanced empathy.")
    print("\nSpecial commands:")
    print("  /debug   - Show debug information including quantum states")
    print("  /reset   - Reset conversation history")
    print("  /quantum - Show quantum classifier info")
    print("  /exit    - End session")
    print("========================================================\n")
    
    user_id = "quantum_user"
    

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()

            # Check for special commands
            if user_input.lower() == "/exit":
                print("Ending session. Take care!")
                # Cleanup before exit
                if hasattr(bot, 'cleanup'):
                    bot.cleanup()
                break
            elif user_input.lower() == "/debug":
                bot.display_debug_info(user_id)
                continue
            elif user_input.lower() == "/clean":
                # Force memory cleanup
                if hasattr(bot, 'cleanup'):
                    bot.cleanup()
                    print("Memory cleaned. Reinitializing...")
                    bot = initialize_quantum_bot(use_dummy_if_failed=True)
                    if not bot:
                        print("Failed to reinitialize. Exiting.")
                        break
                continue
            elif user_input.lower() == "/reset":
                    # Get user-specific memory
                    memory = bot.get_memory(user_id)
                    memory.clear()
                    if user_id in bot.user_emotions:
                        bot.user_emotions[user_id] = []
                    print("Conversation history and emotion tracking have been reset.")

            elif not user_input:
                continue

            # Process the input and get response
            response = bot.chat_with_history(user_input, user_id)

            # Display the response
            print(f"Bot: {response}")

            # Show current emotion after each exchange
            if user_id in bot.user_emotions and bot.user_emotions[user_id]:
                current_emotion = bot.user_emotions[user_id][-1]
                #if current_emotion != "neutral":
                    #print(f"[Detected emotion: {current_emotion}]")

            # Periodically check memory status
            if torch.cuda.is_available() and hasattr(bot, 'model') and bot.classical_model and not bot.is_dummy_mode:
                try:
                    free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
                    if free_memory < 500e6:  # Less than 500MB free
                        print("[Warning: Low GPU memory. Consider using /clean to free up resources]")
                except:
                    pass  # Ignore errors in memory checking

        except KeyboardInterrupt:
            print("\nSession interrupted. Ending session.")
            if hasattr(bot, 'cleanup'):
                bot.cleanup()
            break
        except Exception as e:
            print(f"Error in interactive session: {e}")
            print("Let's continue anyway.")

# Main execution
if __name__ == "__main__":
    try:
        run_quantum_interactive_session()
    finally:
        # Final cleanup before exit
        print("Cleaning up resources...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 